In [ ]:
%pip install --upgrade openai


### Match the closest file name video with the Database from CommonSense that already hold age ratings

In [15]:
import os
import json
import pandas as pd
import openai
from dotenv import load_dotenv

# Load OpenAI API key from .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Assign the actual API key from environment variable
openai.api_key = api_key

# Path to your Excel file
excel_path = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/01_Scrapping_List/DB_Age_OnlyAnalysedVideos.xlsx'

# Load the Excel file
excel_data = pd.read_excel(excel_path)

# Extract relevant columns (adjust the column names as needed)
excel_data = excel_data[['Title', 'Age Rating', 'Age']]  # Adjust as per your actual column names

# Path to the directory containing the JSON files
json_dir = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/1 Done'

# List to store show names from JSON files
json_shows = []

# Iterate through each folder (ignoring non-directory files like .DS_Store)
for show_folder in os.listdir(json_dir):
    folder_path = os.path.join(json_dir, show_folder)
    
    if os.path.isdir(folder_path):  # Only process if it's a directory
        for file_name in os.listdir(folder_path):
            if file_name.endswith('_stats.json'):
                json_file_path = os.path.join(folder_path, file_name)
                with open(json_file_path, 'r') as f:
                    data = json.load(f)
                    json_shows.append(data['video_title'])

# Convert the JSON show names into a DataFrame
json_shows_df = pd.DataFrame(json_shows, columns=['video_title'])

# Function to extract the title from the OpenAI response
def extract_title_from_response(response_content):
    # Assuming the matched title is after "The closest matching title to 'X' is 'Y'."
    return response_content.split("'")[-2] if "'" in response_content else response_content

# Function to use OpenAI API to find closest match (with batching)
def find_closest_match(json_show, excel_titles, batch_size=50):
    matched_title = None
    
    # Break the list of titles into batches to avoid exceeding token limit
    for i in range(0, len(excel_titles), batch_size):
        title_batch = excel_titles[i:i+batch_size]  # Take a batch of titles
        
        prompt = f"Match the following TV show title: '{json_show}' to one of the following: {', '.join(title_batch)}. Return the closest matching title."
        
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an assistant that matches TV show titles."},
                {"role": "user", "content": prompt}
            ]
        )
        
        # Extract the matching title from the response
        if response and response.choices:
            response_content = response.choices[0].message.content.strip()
            matched_title = extract_title_from_response(response_content)
        
        # Debug: Print the OpenAI API response
        print(f"OpenAI API Response for {json_show}: {response}")
        
        # If we find a match, return it
        if matched_title:
            return matched_title

    # If no match is found, return a default message or None
    return matched_title or "No Match Found"

# List to store matches
matched_shows = []

# Match each JSON show title with the Excel titles (using batching)
for json_show in json_shows:
    matched_title = find_closest_match(json_show, excel_data['Title'].tolist(), batch_size=50)  # Adjust batch_size if needed
    matched_shows.append({'json_show': json_show, 'matched_title': matched_title})

# Convert the results into a DataFrame
matched_df = pd.DataFrame(matched_shows)

# Normalize the case and remove extra spaces before merging
matched_df['matched_title'] = matched_df['matched_title'].str.lower().str.strip()
excel_data['Title'] = excel_data['Title'].str.lower().str.strip()

# Merge the matched shows with the Excel data to get age ratings
final_df = pd.merge(matched_df, excel_data, left_on='matched_title', right_on='Title')

# Now you have a DataFrame with JSON show titles and corresponding age ratings
print("Final DataFrame (After Merge with Excel):")
print(final_df)


OpenAI API Response for Molang: ChatCompletion(id='chatcmpl-A89uQi2AsKPCWdX33jIxCBY0W6l3e', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The closest matching title to 'Molang' is 'Molang'.", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726508078, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=14, prompt_tokens=179, total_tokens=193, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))
OpenAI API Response for Duck_And_Goose: ChatCompletion(id='chatcmpl-A89uSzd21yCb7MwuVg6nRf3dZc4dq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The closest matching title to 'Duck_And_Goose' is 'Duck & Goose'.", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726508080, model='gpt-4-0613', object='chat.completion', service_tie

In [16]:
final_df

,json_show,matched_title,Title,Age Rating,Age
0,Molang,molang,molang,age 3+,3
1,Duck_And_Goose,duck & goose,duck & goose,age 2+,2
2,Digimon_Digital_Monsters,digimon: digital monsters,digimon: digital monsters,age 6+,6
3,Tales_of_the_Jedi,avatar: the last airbender,avatar: the last airbender,age 8+,8
4,Grizzy_and_the_Lemmings,grizzy and the lemmings,grizzy and the lemmings,age 6+,6
5,Polly_Pockets,polly pocket,polly pocket,age 5+,5
6,Pat_The_Dog,pat the dog,pat the dog,age 6+,6
7,Supernatural_Academy,mysticons,mysticons,age 7+,7
8,Mysticons,mysticons,mysticons,age 7+,7
9,The_Doc_McStuffins_Christmas_Special,the doc mcstuffins christmas special,the doc mcstuffins christmas special,age 4+,4
